In [ ]:
import pandas as pd
import numpy as np

### Title Basics

In [ ]:
basics = pd.read_csv('title.basics.tsv', sep='\t', dtype={'startYear': str})
basics = basics.drop(columns=['originalTitle', 'endYear', 'genres'])
len(basics)

#### Filter out entries that aren't movies

In [ ]:
is_movie = basics['titleType'] == 'movie'
basics = basics[is_movie]
len(basics)

#### Filter out adult titles

In [ ]:
not_adult = basics['isAdult'] == 0
basics = basics[not_adult]
basics = basics.drop(columns=['isAdult'])
len(basics)

#### Filter out movies that do not have a start year

In [ ]:
has_start_year = basics['startYear'] != '\\N'
basics = basics[has_start_year]
len(basics)

#### Filter out movies before 1939 (arbitrary, but Gone with the Wind and Wizard of Oz came out this year)

In [ ]:
after1938 = basics['startYear'] > '1938'
basics = basics[after1938]
len(basics)

In [ ]:
#basics

### Principals

In [ ]:
principals = pd.read_csv('title.principals.tsv', sep='\t', dtype={'types': str})
len(principals)

#### Only actors or actresses, not director, composer, etc.

In [ ]:
is_actor = (principals['category'] == 'actor') | (principals['category'] == 'actress')
principals = principals[is_actor]
principals = principals.drop(columns=['category'])
len(principals)

#### Remove if they played no characters

In [ ]:
 principals = principals[principals['characters'] != '\\N']
 len(principals)

#### Remove duplicate "actors in movies" records

In [ ]:
principals = principals.drop_duplicates(subset=['nconst', 'tconst'])
len(principals)

### Name Basics

In [ ]:
names = pd.read_csv('name.basics.tsv', sep='\t')

In [ ]:
len(names)

#### Remove names without a birth year

In [ ]:
names = names[names['birthYear'] != '\\N']
len(names)

#### Remove names without a primary profession

In [ ]:
names = names.dropna(subset=['primaryProfession'])
len(names)

#### Remove Names without any known-for titles

In [ ]:
names = names[names['knownForTitles'] != '\\N']
len(names)

#### Drop duplicate names
This will result in removing actors erroneously. For example, if there was a Ben Smith born in 1928 who starred in 2 movies and another Ben Smith born in 1945 who starred in 6 movies, one of those Ben Smiths will be removed from the data set.

This is good enough for now.

In [ ]:
names = names.drop_duplicates(subset=['primaryName'])
len(names)

### Results

In [ ]:
merged = basics.merge(principals, on=['tconst'])
merged = merged.merge(names, on=['nconst'])
merged.columns

In [ ]:
results = pd.DataFrame(
    {'TitleId': merged['tconst'],
     'NameId': merged['nconst'],
     'Movie Title': merged['primaryTitle'],
     'Year': merged['startYear'],
     'Actor': merged['primaryName'],
     'Characters': merged['characters']})
len(results)

In [ ]:
len(pd.unique(results['Actor']))

### Things To Do With The Data

In [ ]:
avengers_movie = (results['Movie Title'] == 'The Avengers') & (results['Year'] == '2012')
avengers_principal_actors = results[avengers_movie]
avengers_principal_actors

In [ ]:
johnny_depp = results['Actor'] == 'Johnny Depp'
johnny_depp_movies = results[johnny_depp]
johnny_depp_movies = johnny_depp_movies.sort_values(by=['Year'])
johnny_depp_movies

#### Write the combined results to a file

In [ ]:
results.to_csv('movies_and_actors.csv')